In [1]:
import pandas as pd

# Cargar los archivos
df_mensual = pd.read_csv("../data/clean/interes_fijo.csv")  # contiene 'Periodo' como '2025M3'
df_trimestral = pd.read_csv("../data/clean/ipv_stnd.csv")        # contiene 'Periodo' como '2025T1'

# ----------- Procesar periodos mensuales -----------
periodos_mes = df_mensual["Periodo"].dropna().unique()
dim_tiempo = []

for p in periodos_mes:
    anio = int(p[:4])
    mes = int(p.split("M")[1])
    fecha = pd.Timestamp(year=anio, month=mes, day=1)
    trimestre = f"T{((mes - 1) // 3) + 1}"
    dim_tiempo.append({
        "periodo_mes": p,
        "periodo_trimestre": f"{anio}{trimestre}",
        "fecha": fecha,
        "anio": anio,
        "mes": mes,
        "trimestre": trimestre
    })

# ----------- Procesar periodos trimestrales -----------
periodos_trim = df_trimestral["Periodo"].dropna().unique()

for p in periodos_trim:
    anio = int(p[:4])
    trimestre = p[-1]
    mes_inicio = {"1": 1, "2": 4, "3": 7, "4": 10}.get(trimestre)
    fecha = pd.Timestamp(year=anio, month=mes_inicio, day=1)
    if not any(d["periodo_trimestre"] == p for d in dim_tiempo):
        dim_tiempo.append({
            "periodo_mes": None,
            "periodo_trimestre": p,
            "fecha": fecha,
            "anio": anio,
            "mes": mes_inicio,
            "trimestre": f"T{trimestre}"
        })

# Convertir a DataFrame y guardar como CSV
df_dim_tiempo = pd.DataFrame(dim_tiempo).sort_values(by="fecha").reset_index(drop=True)
df_dim_tiempo.to_csv("../data/clean/dim_tiempo.csv", index=False)


In [3]:
# Cargar los archivos con información de comunidades y jornadas
df_salario_comunidad = pd.read_csv("../data/clean/salario_ccaa.csv")
df_salario_edad = pd.read_csv("../data/clean/salario_edad.csv")

# ----------- dim_comunidades -----------
comunidades = df_salario_comunidad["Comunidades y Ciudades Autónomas"].dropna().unique()
df_dim_comunidades = pd.DataFrame(sorted(comunidades), columns=["comunidad_autonoma"])
df_dim_comunidades.to_csv("../data/clean/dim_comunidades.csv", index=False)

# ----------- dim_jornada -----------
jornada_1 = df_salario_comunidad["Tipo de jornada"].dropna().unique()
jornada_2 = df_salario_edad["Tipo de jornada"].dropna().unique()
jornadas = sorted(set(jornada_1) | set(jornada_2))
df_dim_jornada = pd.DataFrame(jornadas, columns=["tipo_jornada"])
df_dim_jornada.to_csv("../data/clean/dim_jornada.csv", index=False)
